In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

#TODO: kolommen van int omzetten naar "Int64" zodat np.NaN kan worden opgeslagen ipv 0 als placeholder
#TODO: NaN-types als laatste verwijderen na alle cleanup

# In hoeverre is het mogelijk om de budget-winst verhouding te voorspellen?
Ook voor deze onderzoeksvraag zullen de zeven stappen van het Data Science-proces worden toegepast. 
Stappen 1, 2 en 3 zullen in grote maten hetzelfde zijn als bij de overige twee onderzoeksvragen.
### Het Data Science-proces
1. Data collection
2. Data processing (ook wel data munging)
3. Data cleaning
4. Data exploration & analysis
5. Model building
6. Visualization
7. Communication

## 1. Data Collection
Voor het beantwoorden van deze onderzoeksvraag zal alleen gebruik worden gemaakt van `movie.csv`.
Om te zien of de dataset `movie.csv` goed is ingeladen, worden de eerste vijf rijen getoond:

In [2]:
df_movies = pd.read_csv('../../data/movie.csv')
print(f"In totaal zijn er {len(df_movies)} films.\n")
df_movies.head()

Te zien is dat de data goed wordt ingelezen en de waardes in de juiste kolommen komen te staan.
Verder zijn er al NaN-waardes te zien en staan veel gegevens opgeslagen als floats.

## 2. Data Processing
Ook deze stap is grotendeels voor ons gedaan. De data is goed opgeslagen in een `.csv`-bestand
en kan direct worden opgeslagen als een _Pandas_ DataFrame.

Verder rest ons nog de volgende drie stappen:
1. Het selecteren van de gewenste kolommen
2. Het aanpassen van onduidelijke kolomnamen
3. Het aanpassen van de datatypen

In [3]:
# Selecteren gewenste kolommen
df_movies = df_movies[["movie_title", "budget", "gross", "title_year"]]

#Onduidelijke kolomnamen aanpassen
df_movies.rename(columns={'movie_title': 'Movie title',
                          'budget': 'Budget',
                          'gross': 'Gross',
                          'title_year': 'Release year'}, inplace=True)

# Datatypen aanpassen: Floats naar integers omzetten
df_movies.fillna(0, inplace=True)
df_movies = df_movies.astype({"Budget": int, "Gross": int})


# Datatypes aanpassen: De kolom `Release year` omzettten van integers naar het datetime-datatype
df_movies["Release year"] = pd.to_datetime(df_movies["Release year"], format='%Y', errors='coerce')

df_movies.dtypes

Het DataFrame `df_movies` ziet er nu als volgt uit:

In [4]:
df_movies.head()

Alle kolommen van het DataFrame hebben nu het juiste datatype.
`Budget` en `Gross` bestaan uit integers en `Release year` bestaat uit het `datetime` datatype.
Echter zijn er nog wel 0-waardes en NaT-waardes zichtbaar. Deze rijen dienen tijdens stap 3. Data
Cleaning nog te worden opgeschoont.

## 3. Data Cleaning
1. Het verwijderen van NaN-types
2. Het verwijderen van dubbele `Movie titles`
3. Het omzetten van onrealistische waardes naar 0
4. Het verwijderen van rijen met een budget van 0

In [5]:
# NaN-types verwijderen uit de titels
df_movies.dropna(inplace=True)
print(f"In totaal zijn er {len(df_movies)} films die een titel hebben.")

# Dubbele titels verwijderen
df_movies.sort_values("Release year", inplace=True)  # Sorteren op uitgavejaar
df_movies.drop_duplicates(subset="Movie title", keep="last", inplace=True)  # Alleen meest recente versie blijft bewaard
print(f"In totaal zijn er {len(df_movies)} films zonder duplicaten over.")

# Onrealistische waardes omzetten
print(f"In totaal zijn er {df_movies[['Budget', 'Gross']].lt(0).sum().sum()} films met een negatief Budget of Gross.")
print(f"In totaal zijn er {df_movies[['Budget']].gt(400_000_000).sum().sum()} films met een budget hoger dan de duurste film ooit.")
print(f"In totaal zijn er {df_movies[['Budget', 'Gross']].gt(2_800_000_000).sum().sum()} films met een hogere omzet dan de meest succesvolle film ooit.")

# Negatieve waardes omzetten
neg = df_movies._get_numeric_data()
neg[neg < 0] = 0
# Credits: stackoverflow.com/questions/27759084/how-to-replace-negative-numbers-in-pandas-data-frame-by-zero

# 'Te dure' films verwijderen
df_movies = df_movies[(df_movies['Budget'] < 400_000_000)]

# Verwijderen films met budget van 0
df_movies = df_movies[(df_movies[['Budget', 'Gross']] != 0).all(axis=1)]

Na stap 3. Data Cleaning ziet het DataFrame er als volgt uit:

In [6]:
df_movies.head()

Het DataFrame is nu volledig opgeschoond en klaar voor de volgende stap.
  
NaN-types kunnen niet worden opgeslagen in kolommen met het datatype `int32`*.
Als oplossing werden onder andere de onrealistische waardes opgeslagen als `0` om 
later te kunnen worden verwijderd uit het DataFrame.  
*Bron: pandas.pydata.org/pandas-docs/stable/user_guide/integer_na.html

Het omzetten van alle onrealistische waardes van `Budget` en `Gross` worden vervolgens omgezet naar `0`.
Onrealistische waardes van `Budget` en `Gross` zijn onder andere de
negatieve waardes. Andere onrealistische waardes zijn films met een `Budget` en/of `Gross` van meer dan
2.8 miljard. Er is immers nog nooit een film* geweest die meer dan 2.8 miljard opbracht.
Verder werden alle films met een `Budget` van meer dan 400 miljoen gedropped, de duurste film ooit
kostte niet meer dan 400 miljoen. Door deze stap worden ook enkele extreme outliers gefilterd uit
de data wat de visualisatie ten goede komt.  
Zodra de onrealistische waardes omgezet zijn naar `0` worden deze uit de dataset verwijderd.  
*Bron: nl.wikipedia.org/wiki/Lijst_van_succesvolste_films 

Door het DataFrame te sorteren op `Release year` en vervolgens de laatste waarde te behouden, wordt
de meest recente uitgave van een film bewaard. Te zien is dat maar liefst $(4935-4811=)$ 124 films dubbel
in de dataset voorkwamen.


## 4. Data Exploration & Analysis

Nu de data is geprepareerd, kan begonnen worden aan de eerste verkenning. Door middel van
describe krijgen we in één oogopslag een duidelijk beeld van het DataFrame.

In [7]:
df_movies.describe(include = "all")

Enkele verwachtingen:
1. Het meerendeel van de films zal winst maken
2. Nieuwere films zullen relatief meer winst maken dan oudere films
3. Het budget van relatief oudere films zal lager zijn dan dat van relatief nieuwe films 

Enkele eerste observaties:
- Het DataFrame bevat nog 3778 (74.9%) van de originele 5043 rijen 
- `Movie title` bevat zoals beoogd alleen maar nog unieke waardes.
- Gemiddeld brengt een film 50 miljoen op en is er een budget van 37 miljoen. Echter heeft
 het budget een standaard deviatie van 43 miljoen en de omzet een standaard deviatie van 69 miljoen.
- Het `Release year` 2002 komt met 189x het vaakst voor
- Het DataFrame bevat films die uitgegeven zijn tussen 1920 en 2016. Een interval van bijna 100 jaar.


In de volgende scatterplot zijn het budget en de omzet van films geplot.
Dit geeft een eerste beeld van de spreiding van de data en de relatie tussen het budget en de omzet.
Er wordt een sample van 350 (+- 10%) stuks genomen om de data beter te visualiseren.


In [8]:
df_movies_sample = df_movies.sample(350, random_state=42)
sns.scatterplot("Budget", "Gross", data=df_movies_sample)
plt.axis('square')  # X- en Y-as dezelfde verhouding geven
plt.show()

Een mogelijke functie om, in plaats van een selectie, alle films te analyseren is `jointplot`.
Jointplots geven op een snelle manier de verspreiding maar vooral dichtheid van punten weer.
Er is goed te zien dat de meeste films onder een budget van één miljoen blijven en 
dat veruit de meeste films onder een omzet van twee miljoen blijven.
Er valt zelfs al een kleine positieve correlatie te spotten.

Om de plot overzichtelijk te houden is gekozen om de dichtheid weer te geven.
Het grote aantal films kan, zoals we hierboven al zagen, tot een onoverzichtelijke plot leiden.
Credits: seaborn.pydata.org/generated/seaborn.jointplot.html

In [9]:
sns.jointplot("Budget", "Gross", data=df_movies, kind="kde", space=0)
plt.show()

In de vorige jointplot waren individuele outliers niet te zien.
Boxplots zijn uitermate geschikt om deze wel weer te geven.

De omzet van films is veel verspreider dan het budget van films. Tevens zijn
er bij de omzet veel meer extreme outliers te zien.
Door het grote aantal uitschieters kunnen zelfs boxplots heel onoverzichtelijk worden.

In [10]:
df_movies.boxplot(grid=False)


Een laatste poging om ook de outliers in kaart te brengen wordt gedaan door twee
histogrammen te plotten. Bovenop de histogrammen worden streepjes getekend. Ieder streepje is een
film. De outliers zijn nu zichtbaar geworden. Direct valt te zien dat het budget
twee extreme uitschieters heeft bij drie en resp. vier miljoen.
Gross heeft uitschieters die door gaan tot zo'n ongeveer 8 miljoen.

Ook hier wordt weer duidelijk dat het meerendeel van de films het budget onder de één miljoen houdt
en de opbrengst vooral onder de twee miljoen blijft.

In [11]:
plot, axes = plt.subplots(1, 2)
sns.distplot(df_movies["Budget"], rug=True, kde=False, ax=axes[0])
sns.distplot(df_movies["Gross"], rug=True, kde=False, ax=axes[1], color="r")


Pandas heeft een hele handige methode ingebouwd voor DataFrames. Deze methode berekend direct de
correlatie tussen iedere combinatie van twee variabelen. Standaard wordt de correlatie
door middel van Pearsons correlatiecoëfficiënt berekend.

Volgens Pearsons correlatiecoëfficiënt blijkt dat er een correlatie van 0.63 is tussen Budget en
Gross.

In [12]:
df_movies.corr()

## 5. Model Building

